In [1]:
import numpy as np
np.random.seed(27)
import time as tp
import pandas as pd
import zipfile
import wget
from dataprep.eda import plot, plot_correlation, plot_missing
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from string import punctuation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import os

NumExpr defaulting to 8 threads.


C:\Users\Apurva\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Apurva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data Load

In [2]:
# parameters to be used during DATA LOAD & EDA

# downloading the data directly from url
data_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"
wget.download(data_url,'drugsCom_raw.zip')
# unzipping the contents
zf = zipfile.ZipFile('drugsCom_raw.zip')
# checking the contents
zipfile.ZipFile.namelist(zf)

['drugsComTest_raw.tsv', 'drugsComTrain_raw.tsv']

In [3]:
# reading both the train and test data
train_raw = pd.read_csv( zf.open(zipfile.ZipFile.namelist(zf)[1]) , sep = "\t" )
test_raw = pd.read_csv( zf.open(zipfile.ZipFile.namelist(zf)[0]) , sep = "\t" )
train_raw.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [4]:
# Reviewing the dimention of train the datasets
print ( "train obs: ", train_raw.shape[0],  "\nno of features: ", train_raw.shape[1] )
print (train_raw.dtypes , "\n" , train_raw.describe() )

train obs:  161297 
no of features:  7
Unnamed: 0       int64
drugName        object
condition       object
review          object
rating         float64
date            object
usefulCount      int64
dtype: object 
           Unnamed: 0         rating    usefulCount
count  161297.000000  161297.000000  161297.000000
mean   115923.585305       6.994377      28.004755
std     67004.445170       3.272329      36.403742
min         2.000000       1.000000       0.000000
25%     58063.000000       5.000000       6.000000
50%    115744.000000       8.000000      16.000000
75%    173776.000000      10.000000      36.000000
max    232291.000000      10.000000    1291.000000


In [5]:
# Reviewing the dimention of train the datasets
print ( "test obs: ", test_raw.shape[0],  "\nno of features: ", test_raw.shape[1] )
print (test_raw.dtypes , "\n" , test_raw.describe() )

test obs:  53766 
no of features:  7
Unnamed: 0       int64
drugName        object
condition       object
review          object
rating         float64
date            object
usefulCount      int64
dtype: object 
           Unnamed: 0        rating   usefulCount
count   53766.000000  53766.000000  53766.000000
mean   116386.701187      6.976900     27.989752
std     67017.739881      3.285207     36.172833
min         0.000000      1.000000      0.000000
25%     58272.500000      4.000000      6.000000
50%    116248.500000      8.000000     16.000000
75%    174586.750000     10.000000     36.000000
max    232284.000000     10.000000    949.000000


In [6]:
# checking the content of train & test data
print ( 'No of unique values with unnamed variable:', train_raw['Unnamed: 0'].nunique(), test_raw['Unnamed: 0'].nunique() )
print ( 'Unique values with rating variable:', train_raw['rating'].nunique(), test_raw['rating'].nunique() )

No of unique values with unnamed variable: 161297 53766
Unique values with rating variable: 10 10


## Abstract:
The drug review dataset has 6 variables (excluding the unique identifier 'unnamed').

1. drugName (categorical): name of drug
2. condition (categorical): name of condition
3. review (text): patient review
4. rating (numerical): 10 star patient rating ( 1- 10 )
5. date (date): date of review entry (long format: month dd, yyyy )
6. usefulCount (numerical): number of users who found review useful
The Training data has 160 thousand obervations and test has 50 thousand obervations. Both data has same variables.

# EDA - exploratory data analysis

# <span style="color:blue">  Dependent Variable: rating 

In [23]:
print ( train_raw['rating'].describe() )
print ( test_raw['rating'].describe() )

count    161297.000000
mean          6.994377
std           3.272329
min           1.000000
25%           5.000000
50%           8.000000
75%          10.000000
max          10.000000
Name: rating, dtype: float64
count    53766.000000
mean         6.976900
std          3.285207
min          1.000000
25%          4.000000
50%          8.000000
75%         10.000000
max         10.000000
Name: rating, dtype: float64


In [24]:
plot(train_raw, 'rating' )

In [25]:
plot_correlation(train_raw)

In [27]:
train_raw['rating_3'] = np.where( train_raw['rating'] > 7 , 3 , np.where ( train_raw['rating'] > 3 , 2 , 1) ).astype(object)
plot(train_raw, 'rating_3' )

## Abstract:
The Rating has 10 unique values (1 -10). The mean is ~7 and median ~8, which means that on an average the ratings are good than bad. The frequency distributional shows a bimodal nature, with 2 peaks at 1 & 10, that is customers provides review when either they have a positive or negative feedback. As expected peak at 10 is higher than at 1, hence more number of customer with positive view has provided their feedback. The Distributiuon of rating between train and test is alike. 
It is positively correlated with usefulCount but with low strength (0.23)

On binning the rating variables in 3 categories ( 1-3, 4-7,8-10), the distribution is visibily skewed towards category 3. Hence we need to resample the data for equal representation for all 3 buckets.

# <span style="color:blue">  Feature: drugName

In [20]:
print (round ( 100* train_raw['drugName'].value_counts()[0:20].sum() / len ( train_raw),2), '% of total' )
plot(train_raw , 'drugName', ngroups  = 20)

19.5 % of total


In [21]:
print (round ( 100* test_raw['drugName'].value_counts()[0:20].sum() / len ( test_raw),2), '% of total' )
plot(test_raw , 'drugName', ngroups  = 20 )

13.43 % of total


In [22]:
plot(train_raw,'rating','drugName', ngroups = 20)

In [32]:
plot(train_raw,'drugName', 'rating_3', ngroups = 20)

## Abstract:
Their are 3000+ unique drug names present in the training data. The top 20 is as shown in the above plot which encompasses ~20% of the overall drug review. Distribution across train and test is similiar
The median ratings for top 20 drugs are very different. Hence drugname should be introduced as a factor in the model.
When checked across the 3 bucket rating variable, we see the distribution not to be same across different drugs as before.

# <span style="color:blue">  Feature: condition

In [37]:
print (round ( 100* train_raw['condition'].value_counts()[0:10].sum() / len ( train_raw),2), '% of total' )
plot(train_raw , 'condition', ngroups  = 10)

45.85 % of total


In [38]:
plot(train_raw,'rating','condition', ngroups = 20)

In [39]:
plot(train_raw,'condition', 'rating_3', ngroups = 20)

In [47]:
plot_missing(train_raw, "condition", "rating3",  bins = 3 )

In [63]:
#train_raw[train_raw.condition == np.nan ].rating3.value_counts()
pd.crosstab ( train_raw.condition.isnull()  , train_raw.rating3 , normalize = 'columns')
#train_raw.rating3.value_counts()

rating3,1,2,3
condition,,,
False,0.993868,0.993443,0.994918
True,0.006132,0.006557,0.005082


## Abstract:
Their are 800+ unique condition present in the training data. The top (based on highest frequency) one being Birth Control, which alone factors for ~18% of the training data.
The top 10 condition can be seen in the above plot. It explains 46% of the entire training data, which means that just half the ratings are for other rest 875 conditions.

Different condition are distributed differently across rating either 10 or 3 categories. Since Birthcontrol has large number of observation in the data, it kind of drives the representation of total condition.
We see a merit in introducing the top 10 condition as dummy variable in the model.

0.6% data is missing, which is same across all ratings, it should be safe to remove such rows.